In [1]:
%load_ext autoreload
%autoreload 2

from statsmodels.base.model import GenericLikelihoodModel

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import sys
import re

from scipy.interpolate import make_interp_spline, BSpline
from scipy.stats import multivariate_normal

sys.path.append("../")
import vuong_tests11 as vuong_tests_fast

In [2]:
class JointNormal1(GenericLikelihoodModel):
    
    def loglikeobs(self, params):
        data = np.concatenate([[self.endog],self.exog.transpose()],axis=0)
        mult_rv = stats.multivariate_normal([params[0], 0.0], [[1,0],[0,1]])
        return mult_rv.logpdf(data.transpose())
    
    
class JointNormal2(GenericLikelihoodModel):
    
    def loglikeobs(self, params):
        data = np.concatenate([[self.endog],self.exog.transpose()],axis=0)
        mult_rv = stats.multivariate_normal([0.0, params[0]], [[1,0],[0,1]])
        return mult_rv.logpdf(data.transpose())


def setup_shi(yn,xn):
    # model 1 grad, etc.
    nobs = yn.shape[0]
    model1_param = np.array([yn.mean()])
    model2_param = np.array([xn.mean()])
    
    model1_deriv = JointNormal1(yn,xn)
    ll1 = model1_deriv.loglikeobs(model1_param)
    grad1 =  model1_deriv.score_obs(model1_param).reshape( (nobs,1) )
    hess1 = model1_deriv.hessian(model1_param)
    
    
    model2_deriv = JointNormal2(yn,xn)
    ll2 = model2_deriv.loglikeobs(model2_param)
    grad2 =  model2_deriv.score_obs(model2_param).reshape( (nobs,1) )  
    hess2 = model2_deriv.hessian(model2_param)
    
    return ll1,grad1,hess1,model1_param,ll2,grad2,hess2,model2_param

def gen_data(beta= 1.5, nobs=1000):
    cov = [[25, 0], [0, 1]]
    data = np.random.multivariate_normal([beta,beta], [[25,0],[0,1]],  nobs)
    return data[:,0],data[:,1],nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn)
print(grad1.shape,hess1.shape)
#NOTE! Weird size distortions with shi's test when theta = .5....

(1000, 1) (1, 1)


In [3]:
num_sims = 500
trials =1000
adapt_c = True
data_tuned_epsilon = True
epsilon = .5

In [4]:
nobs =100

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn)
gen_data_ex = lambda : gen_data(nobs=nobs, beta=0)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=.5,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 0.96 & 0.67 & 0.90 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.15 & 0.01 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.03 & 0.18 & 0.08 & 0.00   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([1., 0., 0.]), array([0.956, 0.016, 0.028]), array([0.666, 0.152, 0.182]), array([0.922, 0.024, 0.054]), array([0.956, 0.01 , 0.034]), array([0.904, 0.014, 0.082]), array([1., 0., 0.]), 12.847562969994996, 20.120193381961165, 20.07927854674009)


In [5]:
nobs =200

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn)
gen_data_ex = lambda : gen_data(nobs=nobs, beta=0)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=.5,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 0.96 & 0.68 & 0.93 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.15 & 0.01 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.03 & 0.17 & 0.06 & 0.00   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([1., 0., 0.]), array([0.952, 0.022, 0.026]), array([0.682, 0.152, 0.166]), array([0.94 , 0.016, 0.044]), array([0.96 , 0.012, 0.028]), array([0.93, 0.01, 0.06]), array([1., 0., 0.]), 11.81611285179251, 17.970584981158535, 19.638221353833178)


In [6]:
nobs = 500


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn)
gen_data_ex = lambda : gen_data(nobs=nobs, beta=0)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=.5,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 0.95 & 0.71 & 0.94 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.15 & 0.01 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.03 & 0.14 & 0.05 & 0.00   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([1., 0., 0.]), array([0.94 , 0.018, 0.042]), array([0.712, 0.146, 0.142]), array([0.934, 0.022, 0.044]), array([0.952, 0.014, 0.034]), array([0.938, 0.014, 0.048]), array([1., 0., 0.]), 12.715788375855054, 18.97563144725841, 20.2729628507343)
